In [ ]:
import sagemaker, boto3
from sagemaker.pytorch import PyTorch

job_name = "effnetb4-skin-job-test-2025-08-28-04-02-01"

# 상태만 빠르게 체크
sm = boto3.client("sagemaker")
print(sm.describe_training_job(TrainingJobName=job_name)["TrainingJobStatus"])

# 로그 스트리밍 (기존 잡에 attach)
sess = sagemaker.Session()
est = PyTorch.attach(training_job_name=job_name, sagemaker_session=sess)
est.latest_training_job.wait(logs=True)  # 여기서 CloudWatch 로그가 쭉 출력됨

In [ ]:
from sagemaker.pytorch import PyTorchModel
from sagemaker.serializers import IdentitySerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker import get_execution_role
import time

from sagemaker.pytorch import PyTorchModel

role = get_execution_role()

model = PyTorchModel(
    model_data="s3://say1-5team-bucket/densenet-output/effnetb4-skin-job-test-2025-08-28-04-02-01/output/model.tar.gz",
    role=role,
    entry_point="inference.py",
    source_dir=".",
    framework_version="2.1",
    py_version="py310",
)

endpoint_name = f"say1-5team-skin-endpoint-{time.strftime('%Y-%m-%d-%H-%M-%S')}"
predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.g4dn.xlarge",
    endpoint_name=endpoint_name,
    tags=[{"Key":"project","Value":"pre-5team"}],
)

predictor.serializer = IdentitySerializer("image/jpeg")
predictor.deserializer = JSONDeserializer()
print("✅ Endpoint:", predictor.endpoint_name)
